In [1]:
%pip install turicreate

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.


In [2]:
import json
import pandas as pd
import glob
import os
import turicreate as tc
import shutil
import numpy as np

#os.remove('/Users/marcoincerti/Desktop/dataML/Data_ready.csv')

shutil.rmtree(r'/Users/marcoincerti/Desktop/change-floor-detection-ActivityClassifier-CoreMLModel/dataML/train')
shutil.rmtree(r'/Users/marcoincerti/Desktop/change-floor-detection-ActivityClassifier-CoreMLModel/dataML/test')

In [3]:
data = json.load(open("/Users/marcoincerti/Desktop/change-floor-detection-ActivityClassifier-CoreMLModel/dataML/2022-08-22 12:47:18.880 50Hz 26.5s.json"))
data = data['motionRecords']

prev_relative = data[0]['altitude']['relativeAltitude']
data[0]['altitude']['relativeAltitude'] = 0.0
    
prev_pressure = data[0]['altitude']['pressure']
data[0]['altitude']['pressure'] = 0.0
    
for d in range(1, len(data)):
  
    tmp = data[d]['altitude']['relativeAltitude']
    tmp_pressure = data[d]['altitude']['pressure']
    if tmp - prev_relative == 0:
        data[d]['altitude']['relativeAltitude'] = data[d -1]['altitude']['relativeAltitude']
    else:
        data[d]['altitude']['relativeAltitude'] = float(tmp - prev_relative)
        
    if tmp - prev_relative == 0:
        data[d]['altitude']['pressure'] = data[d -1]['altitude']['pressure']
    else:
        data[d]['altitude']['pressure'] = float(tmp_pressure - prev_pressure)
    prev_relative = tmp
    prev_pressure = tmp_pressure
    
df = pd.json_normalize(data, sep='_')
df.drop(columns=['recordTimestamp', 'altitude_pressure','attitude_roll', 'attitude_pitch', 'attitude_yaw'])
df.rename(columns = {'accelerationWithOutGravity_x':'acceleration_x', 'accelerationWithOutGravity_y':'acceleration_y', 'accelerationWithOutGravity_z':'acceleration_z',"altitude_relativeAltitude": "relativeAltitude"}, inplace = True)
df.to_csv(r'/Users/marcoincerti/Desktop/change-floor-detection-ActivityClassifier-CoreMLModel/my_data.csv', index=False)
    

In [3]:
i = 0
df_list = []
for curr_file in glob.glob("/Users/marcoincerti/Desktop/change-floor-detection-ActivityClassifier-CoreMLModel/Dataset/50Hz/climb/*s.json"):
    # Opening JSON file
  with open(curr_file) as json_file:
    data = json.load(json_file)
    
    data = data['motionRecords']
    tmp = data
    while len(data) > 1 and data[0]['altitude']['relativeAltitude'] != 0:
       data.pop(0)
        
    if len(data) < 10:
        data = tmp 
        
    prev_relative = data[0]['altitude']['relativeAltitude']
    data[0]['altitude']['relativeAltitude'] = 0.0
    
    prev_pressure = data[0]['altitude']['pressure']
    data[0]['altitude']['pressure'] = 0.0
    
    for d in range(1, len(data)):
        tmp = data[d]['altitude']['relativeAltitude']
        tmp_pressure = data[d]['altitude']['pressure']
        if tmp - prev_relative == 0:
            data[d]['altitude']['relativeAltitude'] = data[d -1]['altitude']['relativeAltitude']
        else:
            data[d]['altitude']['relativeAltitude'] = float(tmp - prev_relative)
        if tmp - prev_relative == 0:
            data[d]['altitude']['pressure'] = data[d -1]['altitude']['pressure']
        else:
            data[d]['altitude']['pressure'] = float(tmp_pressure - prev_pressure)
        prev_relative = tmp
        prev_pressure = tmp_pressure
    
    df = pd.json_normalize(data, sep='_') 
    
    df['sessionId'] = i
    i +=1
    df['activity'] = "climb"
    df_list.append(df)
    
i = 0
# Opening JSON file
for curr_file in glob.glob("/Users/marcoincerti/Desktop/change-floor-detection-ActivityClassifier-CoreMLModel/Dataset/50Hz/stay/*s.json"):
    # Opening JSON file
  with open(curr_file) as json_file:
    data = json.load(json_file)

    data = data['motionRecords']
    
    tmp = data
    while len(data) > 1 and data[0]['altitude']['relativeAltitude'] != 0:
       data.pop(0)
        
    if len(data) < 10:
        data = tmp 
        
    prev_relative = data[0]['altitude']['relativeAltitude']
    data[0]['altitude']['relativeAltitude'] = 0.0
    
    prev_pressure = data[0]['altitude']['pressure']
    data[0]['altitude']['pressure'] = 0.0
    
    for d in range(1, len(data)):
        tmp = data[d]['altitude']['relativeAltitude']
        tmp_pressure = data[d]['altitude']['pressure']
        if tmp - prev_relative == 0:
            data[d]['altitude']['relativeAltitude'] = data[d -1]['altitude']['relativeAltitude']
        else:
            data[d]['altitude']['relativeAltitude'] = float(tmp - prev_relative)
        if tmp - prev_relative == 0:
            data[d]['altitude']['pressure'] = data[d -1]['altitude']['pressure']
        else:
            data[d]['altitude']['pressure'] = float(tmp_pressure - prev_pressure)
        prev_relative = tmp
        prev_pressure = tmp_pressure

    df = pd.json_normalize(data, sep='_')
    df['sessionId'] = i
    i +=1
    df['activity'] = "stay"
  
    df_list.append(df)
    
i = 0
# Opening JSON file
for curr_file in glob.glob("/Users/marcoincerti/Desktop/change-floor-detection-ActivityClassifier-CoreMLModel/Dataset/50Hz/descend/*s.json"):
    # Opening JSON file
  with open(curr_file) as json_file:
    data = json.load(json_file)
    
    data = data['motionRecords']
    
    tmp = data
    while len(data) > 1 and data[0]['altitude']['relativeAltitude'] != 0:
       data.pop(0)
        
    if len(data) < 10:
        data = tmp 
        
    prev_relative = data[0]['altitude']['relativeAltitude']
    data[0]['altitude']['relativeAltitude'] = 0.0
    
    prev_pressure = data[0]['altitude']['pressure']
    data[0]['altitude']['pressure'] = 0.0
    
    for d in range(1, len(data)):
        tmp = data[d]['altitude']['relativeAltitude']
        tmp_pressure = data[d]['altitude']['pressure']
        if tmp - prev_relative == 0:
            data[d]['altitude']['relativeAltitude'] = data[d -1]['altitude']['relativeAltitude']
        else:
            data[d]['altitude']['relativeAltitude'] = float(tmp - prev_relative)
        if tmp - prev_relative == 0:
            data[d]['altitude']['pressure'] = data[d -1]['altitude']['pressure']
        else:
            data[d]['altitude']['pressure'] = float(tmp_pressure - prev_pressure)
        prev_relative = tmp
        prev_pressure = tmp_pressure
    
    df = pd.json_normalize(data, sep='_')
    df['sessionId'] = i
    i +=1
    df['activity'] = "descend"
    df_list.append(df)
    


i = 0
# Opening JSON file
for curr_file in glob.glob("/Users/marcoincerti/Desktop/change-floor-detection-ActivityClassifier-CoreMLModel/Dataset/50Hz/elevator_up/*s.json"):
    # Opening JSON file
  with open(curr_file) as json_file:
    data = json.load(json_file)
 
    # for reading nested data [0] represents
    # the index value of the list
    data = data['motionRecords']
    
    tmp = data
    while len(data) > 1 and data[0]['altitude']['relativeAltitude'] != 0:
       data.pop(0)
        
    if len(data) < 10:
        data = tmp 
        
    prev_relative = data[0]['altitude']['relativeAltitude']
    data[0]['altitude']['relativeAltitude'] = 0.0
    
    prev_pressure = data[0]['altitude']['pressure']
    data[0]['altitude']['pressure'] = 0.0
    
    for d in range(1, len(data)):
        tmp = data[d]['altitude']['relativeAltitude']
        tmp_pressure = data[d]['altitude']['pressure']
        if tmp - prev_relative == 0:
            data[d]['altitude']['relativeAltitude'] = data[d -1]['altitude']['relativeAltitude']
        else:
            data[d]['altitude']['relativeAltitude'] = float(tmp - prev_relative)
        if tmp - prev_relative == 0:
            data[d]['altitude']['pressure'] = data[d -1]['altitude']['pressure']
        else:
            data[d]['altitude']['pressure'] = float(tmp_pressure - prev_pressure)
        prev_relative = tmp
        prev_pressure = tmp_pressure
        
    df = pd.json_normalize(data, sep='_')
    df['sessionId'] = i
    i +=1
    df['activity'] = "elevator_up"
    df_list.append(df)
    

df_final = pd.concat(df_list)
print(df_final.count())
#df_final = df_final.replace(r'^\s*$', np.nan, regex=True)
df_final = df_final.dropna()
#df_final['altitude_relativeAltitude'].mask(df_final['altitude_relativeAltitude'] < 0.001, 0, inplace=True)
df_final['altitude_relativeAltitude'] = df_final['altitude_relativeAltitude'].replace([0],float(0.001))
df_final['altitude_pressure'] = df_final['altitude_pressure'].replace([0],float(0.0001))
#df_final.drop(df_final[df_final.altitude_relativeAltitude == 0].index, inplace=True)
print(df_final.count())
df_final.head()

recordTimestamp                 25998
attitude_roll                   25998
attitude_pitch                  25998
attitude_yaw                    25998
altitude_pressure               25998
altitude_relativeAltitude       25998
rotationRate_x                  25998
rotationRate_y                  25998
rotationRate_z                  25998
accelerationWithOutGravity_x    25998
accelerationWithOutGravity_y    25998
accelerationWithOutGravity_z    25998
sessionId                       25998
activity                        25998
dtype: int64
recordTimestamp                 25998
attitude_roll                   25998
attitude_pitch                  25998
attitude_yaw                    25998
altitude_pressure               25998
altitude_relativeAltitude       25998
rotationRate_x                  25998
rotationRate_y                  25998
rotationRate_z                  25998
accelerationWithOutGravity_x    25998
accelerationWithOutGravity_y    25998
accelerationWithOutGravity_z    25998

,recordTimestamp,attitude_roll,attitude_pitch,attitude_yaw,altitude_pressure,altitude_relativeAltitude,rotationRate_x,rotationRate_y,rotationRate_z,accelerationWithOutGravity_x,accelerationWithOutGravity_y,accelerationWithOutGravity_z,sessionId,activity
0,1.663156e+09,-0.052272,0.434462,0.049181,0.0001,0.001,-0.221389,-0.386129,0.483542,-0.149001,-0.147727,-0.872737,0,climb
1,1.663156e+09,-0.055845,0.435909,0.058607,0.0001,0.001,0.386478,0.361325,0.342213,-0.211732,-0.113502,-0.788942,0,climb
2,1.663156e+09,-0.049815,0.446139,0.064611,0.0001,0.001,0.612089,0.444514,0.153087,-0.250444,-0.075060,-0.577897,0,climb
3,1.663156e+09,-0.041424,0.458423,0.066811,0.0001,0.001,0.613279,0.495299,-0.012411,-0.263783,-0.036057,-0.332854,0,climb
4,1.663156e+09,-0.030812,0.470435,0.065259,0.0001,0.001,0.603573,0.500283,-0.158583,-0.281347,0.058438,-0.166410,0,climb


In [4]:
# Column name mappings
cols = {
    #"recordTimestamp": "timestamp",
    #"attitude_roll": "attitude_roll",
    #"attitude_pitch": "attitude_pitch",
    #"attitude_yaw": "attitude_yaw",
    "altitude_pressure": "altitude_pressure",
    "altitude_relativeAltitude": "relativeAltitude",
    "rotationRate_x": "rotationRate_x",
    "rotationRate_y": "rotationRate_y",
    "rotationRate_z": "rotationRate_z",
    "accelerationWithOutGravity_x": "acceleration_x",
    "accelerationWithOutGravity_y": "acceleration_y",
    "accelerationWithOutGravity_z": "acceleration_z",
    "sessionId": "session_id",
    "activity": "activity"
    }

csv_cols = ["rotationRate_x", "rotationRate_y", "rotationRate_z", "acceleration_x", "acceleration_y",
            "acceleration_z","relativeAltitude", "altitude_pressure" ]

# Load csv data and rename columns
sf = tc.SFrame(data=df_final[list(cols.keys())]).rename(cols)

# Remove missing activies
sf = sf[sf['activity'] != '']
acts = sf['activity'].unique()

# Split data into training and testing sets
#train, test = sf.random_split(0.8)
train, test = tc.activity_classifier.util.random_split_by_session(sf, session_id='session_id', fraction=0.8)

# Write out training data
path = "train/"
os.mkdir(path)
for a in acts:
  # Check for folder
  cls_path = path + a + "/"
  if not os.path.exists(cls_path):
    os.mkdir(cls_path)
# Split data by activity & session and write to file
  sf_act = train[train['activity'] == a]
  for s in sf_act['session_id'].unique():
    sf_act[sf_act['session_id'] == s][csv_cols].save(cls_path + str(a)+ "_" + str(s) + ".csv")
# Write out testing data
path = "test/"
os.mkdir(path)
for a in acts:
  # Check for folder
  cls_path = path + a + "/"
  if not os.path.exists(cls_path):
    os.mkdir(cls_path)
# Split data by activity & session and write to file
  sf_act = test[test['activity'] == a]
  for s in sf_act['session_id'].unique():
    sf_act[sf_act['session_id'] == s][csv_cols].save(cls_path + str(a) + "_" + str(s) + ".csv")
